In [70]:
import numpy as np
import pickle
import copy
file = "cifar_cnn_users[100]_rounds[100]_frac[1.0]_iid[1]_local_ep[10]_local_bs[10]_attck_frac[0.5]_grads.pkl"
with open(file,"rb") as f:
    data = pickle.load(f)

In [78]:
#Calculate eucledian distance matrix for a given round and layer
def get_round_euclidean_distance_matrix_per_layer(data,layer):
    matrix = [[0 for i in range(len(data))] for i in range(len(data))]
    for user1,(X1,y1) in data.items():
        for user2,(X2,y2) in data.items():
            dist = np.linalg.norm((X1[layer] - (X2[layer])))
            matrix[user1][user2] = round(dist,4)
    return matrix

#Calculate cosine similarity matrix for a given round and layer
def get_round_cos_sim_matrix_per_layer(data,layer):
    matrix = [[0 for i in range(len(data))] for i in range(len(data))]
    for user1,(X1,y1) in data.items():
        for user2,(X2,y2) in data.items():
            x1 = np.array(X1[layer]).flatten()
            x2 = np.array(X2[layer]).flatten()
            norm_product = np.linalg.norm(x1) * np.linalg.norm(x2)
            cos_sim = np.dot(x1,x2)/ norm_product if norm_product != 0 else 0
            matrix[user1][user2] = round(cos_sim,4)
    return matrix

#Calculate angle matrix for a given round and layer
def get_round_angle_matrix_per_layer(data,layer):
    matrix = [[0 for i in range(len(data))] for i in range(len(data))]
    for user1,(X1,y1) in data.items():
        for user2,(X2,y2) in data.items():
            g1 = np.array(X1[layer]).flatten()
            g2 = np.array(X2[layer]).flatten()
            if user1 != user2:
                unit_vector_1 = g1 / np.linalg.norm(g1)
                unit_vector_2 = g2 / np.linalg.norm(g2)
                dot_product = np.dot(unit_vector_1, unit_vector_2)
                radians = np.arccos(dot_product)
                #angle = np.degrees(radians)
                matrix[user1][user2] = round(radians,4)
            else:
                matrix[user1][user2] = 0.0
    return matrix

#Flatten all gradient layers in a given round
def flatten_layers(data):
    X_all = {}
    for user,(X,y) in data.items():
        X_flat = np.array([])
        for layer in X:
            cur = np.array(X[layer]).flatten()
            X_flat = np.concatenate((X_flat,cur),axis=0)
        X_all[user] = [copy.deepcopy(X_flat),y]
    return X_all

#Calculate angle matrix for a given round
def flattened_round_angle_matrix(data):
    matrix = [[0 for i in range(len(data))] for i in range(len(data))]
    for idx1,(g1,y1) in data.items():
        for idx2,(g2,y2) in  data.items():
            if not np.array_equal(g1,g2):
                unit_vector_1 = g1 / np.linalg.norm(g1) if np.linalg.norm(g1) != 0 else 0
                unit_vector_2 = g2 / np.linalg.norm(g2) if np.linalg.norm(g2) != 0 else 0
                dot_product = np.dot(unit_vector_1, unit_vector_2)
                radians = np.arccos(dot_product)
                #angle = np.degrees(radians)
                matrix[idx1][idx2] = round(radians,4)
            else:
                matrix[idx1][idx2] = 0.0
    return matrix

#Calculate per layer angle feature for every device gradient against the global gradient
#i.e. theta_1, theta_2 corresponding with layer_1 and layer_2 angles against the global gradient
def get_round_angles_against_global_model(data,global_gradient):
    #Extract layers
    layers = list(global_gradient.keys())
    matrix = [[0 for i in range(len(layers))] for i in range(len(data))]
    for idx1,(X,y) in data.items():
        for idx2,layer in enumerate(layers):
            g1 = np.array(X[layer]).flatten()
            g2 = np.array(global_gradient[layer]).flatten()
            if not np.array_equal(g1, g2):
                unit_vector_1 = g1 / np.linalg.norm(g1) if np.linalg.norm(g1) != 0 else 0
                unit_vector_2 = g2 / np.linalg.norm(g2) if np.linalg.norm(g2) != 0 else 0
                dot_product = np.dot(unit_vector_1, unit_vector_2)
                radians = np.arccos(dot_product)
                #angle = np.degrees(radians)
                matrix[idx1][idx2] = round(radians,4)
            else:
                 matrix[idx1][idx2] = 0
    return matrix

In [7]:
#Grads data structure
user_angles, global_round_models = {},{}
layers = []

#Grab all layers from a sample gradient (Assumes all gradients have similar layers and shapes)
for layer in data[r][0][0].keys():
    layers.append(layer)

#Iterate through all rounds to compute global FedAvg gradient
#print("\nGrads datastructure\n")
#Iterate through users/participants in current round
for r in range(len(data)):
    #print("round: ",r)
    global_round_models[r] = {layer:[] for layer in layers}
    for idx,val in data[r].items():
        #print("\t: ",idx,"\t:", val[0].keys(),"\tConv1: ",val[0]["conv1"].shape, "\tConv2: ",val[0]["conv2"].shape,"\tlabel:",val[1])
        if idx not in user_angles:
            user_angles[idx] = []
        #Collect all layers of a round into a list
        for layer in layers:
            global_round_models[r][layer].append(val[0][layer])
    
    #Grab number of participants in current round
    n = len(data[r])
    #Avg over all layers in a round
    for layer in layers:
        global_round_models[r][layer] = np.array([sum(x)/n for x in zip(*global_round_models[r][layer])])


print("\nGlobal model\n")
for r,d in global_round_models.items():
    print("round: ",r)
    for layer,val in d.items():
        print("\tlayer: ",layer,"\tShape: ",val.shape)
#         total[layer] = np.array([sum(x) for x in zip(*val)])

# 
# for k,v in total.items():
#     print("\tlayer: ",k,"\tgrads:", (v.shape))
        


Global model

round:  0
	layer:  conv1 	Shape:  (6, 3, 5, 5)
	layer:  conv2 	Shape:  (16, 6, 5, 5)
round:  1
	layer:  conv1 	Shape:  (6, 3, 5, 5)
	layer:  conv2 	Shape:  (16, 6, 5, 5)
round:  2
	layer:  conv1 	Shape:  (6, 3, 5, 5)
	layer:  conv2 	Shape:  (16, 6, 5, 5)
round:  3
	layer:  conv1 	Shape:  (6, 3, 5, 5)
	layer:  conv2 	Shape:  (16, 6, 5, 5)
round:  4
	layer:  conv1 	Shape:  (6, 3, 5, 5)
	layer:  conv2 	Shape:  (16, 6, 5, 5)
round:  5
	layer:  conv1 	Shape:  (6, 3, 5, 5)
	layer:  conv2 	Shape:  (16, 6, 5, 5)
round:  6
	layer:  conv1 	Shape:  (6, 3, 5, 5)
	layer:  conv2 	Shape:  (16, 6, 5, 5)
round:  7
	layer:  conv1 	Shape:  (6, 3, 5, 5)
	layer:  conv2 	Shape:  (16, 6, 5, 5)
round:  8
	layer:  conv1 	Shape:  (6, 3, 5, 5)
	layer:  conv2 	Shape:  (16, 6, 5, 5)
round:  9
	layer:  conv1 	Shape:  (6, 3, 5, 5)
	layer:  conv2 	Shape:  (16, 6, 5, 5)
round:  10
	layer:  conv1 	Shape:  (6, 3, 5, 5)
	layer:  conv2 	Shape:  (16, 6, 5, 5)
round:  11
	layer:  conv1 	Shape:  (6, 3, 5, 5)
	

In [297]:
#print (user_angles)
matrix = [[0 for i in range(2)] for i in range(5)]
print (matrix)
A = [[2,1],[4,5],[4,5]]
print([sum(x) for x in zip(*A)])
print([sum(x)/2 for x in zip(*A)])

[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[10, 11]
[5.0, 5.5]


In [114]:
#Global model
X, y = [],[]
n = len(data)
for r in range(len(data)):
    print("round: ",r)
    matrix = get_round_angles_against_global_model(data[r],global_round_models[r])
    for i,row in enumerate(matrix):
        #del row[i]
        if r == 0:
            X.append(row)
        else:
            #Use exponentially weighted average to calculate theta
            temp_row = []
            for j,_ in enumerate(row):
                theta = r/(r+1) * X[len(X) - n][j] + 1/(r+1) * row[j] 
                temp_row.append(round(theta,4))
            X.append(temp_row)
        y.append(data[r][i][1])
        print("\t",X[-1],"\t",y[-1])

round:  0
	 [0.7907, 0.9645] 	 1
	 [1.18, 1.2414] 	 0
	 [0.83, 1.0069] 	 1
	 [2.0449, 1.3725] 	 0
	 [0.8115, 1.2815] 	 1
	 [0.811, 1.0792] 	 1
	 [0.5949, 1.0776] 	 1
	 [0.5852, 1.3371] 	 1
	 [0.463, 0.98] 	 1
	 [1.9686, 1.8004] 	 0
	 [1.3282, 1.2093] 	 0
	 [0.7021, 1.0981] 	 1
	 [1.7363, 1.6818] 	 1
	 [0.635, 1.1756] 	 0
	 [0.6216, 1.0294] 	 1
	 [0.892, 1.3104] 	 0
	 [1.541, 1.2957] 	 1
	 [0.7167, 0.9259] 	 0
	 [2.2763, 1.9325] 	 1
	 [0.9399, 1.2714] 	 1
	 [2.2225, 1.6622] 	 0
	 [0.6755, 1.0009] 	 0
	 [0.8993, 1.2986] 	 0
	 [1.1031, 1.7245] 	 0
	 [2.4925, 1.9483] 	 0
	 [1.0739, 1.1864] 	 0
	 [1.1215, 1.3648] 	 1
	 [1.233, 1.1453] 	 1
	 [0.805, 1.044] 	 1
	 [1.7612, 1.8155] 	 0
	 [1.2975, 1.3266] 	 1
	 [2.0089, 1.8367] 	 0
	 [1.3108, 1.1519] 	 1
	 [0.8035, 1.0274] 	 0
	 [1.2681, 1.3124] 	 1
	 [0.9874, 1.423] 	 0
	 [0.5633, 0.9014] 	 1
	 [1.4134, 1.4086] 	 0
	 [0.7628, 1.4736] 	 1
	 [1.9053, 1.6387] 	 0
	 [0.7004, 1.1976] 	 1
	 [1.6231, 1.7011] 	 1
	 [1.1738, 1.3369] 	 0
	 [2.2658, 1.568

	 [1.1641, 1.5091] 	 1
	 [1.2321, 1.4374] 	 1
	 [1.2859, 1.6028] 	 0
	 [1.4764, 1.456] 	 0
	 [1.4082, 1.4296] 	 1
	 [1.4965, 1.501] 	 1
	 [1.3042, 1.4297] 	 0
	 [1.1708, 1.3984] 	 1
	 [1.1998, 1.389] 	 0
	 [1.4798, 1.4012] 	 1
	 [1.615, 1.5506] 	 0
	 [1.6252, 1.7065] 	 1
	 [1.3015, 1.4991] 	 1
	 [1.5068, 1.6465] 	 0
	 [1.3721, 1.2888] 	 0
	 [1.1844, 1.4517] 	 0
	 [1.3132, 1.267] 	 0
	 [1.5621, 1.5761] 	 0
	 [1.1291, 1.4562] 	 0
	 [1.5061, 1.5774] 	 1
	 [1.4833, 1.5193] 	 1
	 [1.5134, 1.2314] 	 1
	 [1.5097, 1.7388] 	 0
	 [1.365, 1.3416] 	 1
	 [1.5198, 1.429] 	 0
	 [1.4347, 1.4465] 	 1
	 [1.4074, 1.2989] 	 0
	 [1.5099, 1.5402] 	 1
	 [1.1173, 1.4162] 	 0
	 [1.2186, 1.4364] 	 1
	 [1.5725, 1.3572] 	 0
	 [1.4472, 1.643] 	 1
	 [1.6345, 1.5206] 	 0
	 [1.2343, 1.1774] 	 1
	 [1.3235, 1.3691] 	 1
	 [1.4611, 1.5721] 	 0
	 [1.7158, 1.5725] 	 1
	 [1.3661, 1.6065] 	 0
	 [1.2398, 1.2685] 	 0
	 [1.591, 1.4187] 	 0
	 [1.7274, 1.5497] 	 1
	 [1.3983, 1.6154] 	 1
	 [1.4594, 1.6581] 	 0
	 [1.2505, 1.357] 	 

	 [1.2687, 1.3789] 	 0
	 [1.617, 1.4418] 	 0
	 [1.5461, 1.4854] 	 1
	 [1.3107, 1.4812] 	 1
	 [1.4467, 1.6312] 	 0
	 [1.298, 1.3953] 	 1
	 [1.2571, 1.4376] 	 1
	 [1.2701, 1.3044] 	 0
	 [1.4563, 1.5433] 	 1
	 [1.3085, 1.485] 	 1
	 [1.3571, 1.4848] 	 1
	 [1.2215, 1.4116] 	 0
	 [1.3522, 1.5544] 	 0
	 [1.4657, 1.4589] 	 0
	 [1.4045, 1.546] 	 0
	 [1.4979, 1.5231] 	 0
	 [1.5442, 1.6046] 	 1
	 [1.1054, 1.3584] 	 0
	 [1.4515, 1.5654] 	 0
	 [1.3908, 1.3943] 	 0
	 [1.3198, 1.4193] 	 1
	 [1.2426, 1.4332] 	 0
	 [1.421, 1.5479] 	 1
	 [1.3668, 1.4104] 	 1
	 [1.6056, 1.4903] 	 0
	 [1.3223, 1.4721] 	 1
	 [1.5138, 1.393] 	 0
	 [1.2546, 1.5414] 	 1
	 [1.3444, 1.4516] 	 0
	 [1.3478, 1.5102] 	 1
	 [1.4802, 1.6201] 	 0
	 [1.3326, 1.4689] 	 1
	 [1.4275, 1.395] 	 1
	 [1.3046, 1.535] 	 0
	 [1.4297, 1.417] 	 1
	 [1.1321, 1.3908] 	 0
	 [1.3891, 1.4875] 	 0
	 [1.4273, 1.4147] 	 0
	 [1.4635, 1.4404] 	 0
	 [1.2862, 1.4005] 	 0
	 [1.3605, 1.4559] 	 0
	 [1.5405, 1.4308] 	 1
	 [1.4273, 1.4452] 	 1
	 [1.3373, 1.4757] 	

	 [1.4008, 1.411] 	 1
	 [1.2734, 1.5125] 	 0
	 [1.3218, 1.4026] 	 1
	 [1.0809, 1.3957] 	 0
	 [1.3072, 1.4876] 	 0
	 [1.3845, 1.4583] 	 0
	 [1.3738, 1.4539] 	 0
	 [1.2207, 1.3792] 	 0
	 [1.3002, 1.4619] 	 0
	 [1.4299, 1.4283] 	 1
	 [1.4335, 1.4931] 	 1
	 [1.2487, 1.4274] 	 1
	 [1.2743, 1.4455] 	 1
	 [1.3617, 1.3963] 	 0
	 [1.0764, 1.4567] 	 0
	 [1.2198, 1.5509] 	 0
	 [1.4054, 1.4823] 	 1
	 [1.3955, 1.4657] 	 1
	 [1.1881, 1.4252] 	 0
	 [1.322, 1.5506] 	 1
	 [1.4906, 1.6008] 	 1
	 [1.443, 1.5137] 	 1
	 [1.3816, 1.595] 	 0
round:  13
	 [1.4118, 1.3974] 	 1
	 [1.3011, 1.466] 	 0
	 [1.3428, 1.4274] 	 1
	 [1.2362, 1.4162] 	 0
	 [1.4614, 1.2787] 	 1
	 [1.1878, 1.3985] 	 1
	 [1.2537, 1.3537] 	 1
	 [1.1419, 1.4263] 	 1
	 [1.197, 1.3992] 	 1
	 [1.2975, 1.594] 	 0
	 [1.4221, 1.4405] 	 0
	 [1.3735, 1.5384] 	 1
	 [1.3588, 1.4242] 	 1
	 [1.2166, 1.488] 	 0
	 [1.1797, 1.4364] 	 1
	 [1.2295, 1.4575] 	 0
	 [1.2577, 1.5248] 	 1
	 [1.3071, 1.4864] 	 0
	 [1.3785, 1.5184] 	 1
	 [1.252, 1.4833] 	 1
	 [1.3507

	 [1.116, 1.4849] 	 0
	 [1.2292, 1.5299] 	 0
	 [1.4015, 1.4483] 	 1
	 [1.3408, 1.4504] 	 1
	 [1.2373, 1.4416] 	 0
	 [1.2895, 1.4727] 	 1
	 [1.4485, 1.5326] 	 1
	 [1.4632, 1.4553] 	 1
	 [1.3088, 1.5444] 	 0
round:  17
	 [1.3736, 1.4043] 	 1
	 [1.2675, 1.4955] 	 0
	 [1.3634, 1.4484] 	 1
	 [1.2606, 1.4164] 	 0
	 [1.4413, 1.2941] 	 1
	 [1.2282, 1.3755] 	 1
	 [1.2959, 1.3911] 	 1
	 [1.1449, 1.3963] 	 1
	 [1.171, 1.3919] 	 1
	 [1.2678, 1.571] 	 0
	 [1.4145, 1.4879] 	 0
	 [1.3618, 1.5463] 	 1
	 [1.4018, 1.4388] 	 1
	 [1.275, 1.5154] 	 0
	 [1.2491, 1.4869] 	 1
	 [1.2415, 1.4675] 	 0
	 [1.2885, 1.5304] 	 1
	 [1.341, 1.5142] 	 0
	 [1.3686, 1.5128] 	 1
	 [1.2456, 1.4428] 	 1
	 [1.321, 1.5013] 	 0
	 [1.3201, 1.4538] 	 0
	 [1.21, 1.4904] 	 0
	 [1.3647, 1.4504] 	 0
	 [1.3274, 1.475] 	 0
	 [1.2179, 1.5227] 	 0
	 [1.4237, 1.4594] 	 1
	 [1.3456, 1.468] 	 1
	 [1.3771, 1.3798] 	 1
	 [1.3233, 1.5627] 	 0
	 [1.317, 1.4067] 	 1
	 [1.3579, 1.4842] 	 0
	 [1.5177, 1.5223] 	 1
	 [1.2408, 1.3989] 	 0
	 [1.321, 1

...

	 [1.3926, 1.469] 	 1
	 [1.3484, 1.4125] 	 1
	 [1.295, 1.5175] 	 0
	 [1.2998, 1.4357] 	 1
	 [1.3429, 1.4817] 	 0
	 [1.3662, 1.4908] 	 0
	 [1.4256, 1.4944] 	 0
	 [1.3455, 1.4499] 	 1
	 [1.3645, 1.4385] 	 1
	 [1.3731, 1.5169] 	 0
	 [1.3497, 1.436] 	 1
	 [1.4489, 1.5046] 	 1
	 [1.3023, 1.4832] 	 0
	 [1.3389, 1.4562] 	 1
	 [1.3926, 1.4821] 	 1
	 [1.4099, 1.4419] 	 1
	 [1.3079, 1.4963] 	 0
	 [1.3827, 1.5039] 	 0
	 [1.3413, 1.4742] 	 0
	 [1.293, 1.5253] 	 0
	 [1.3726, 1.5247] 	 0
	 [1.4067, 1.471] 	 1
	 [1.331, 1.5359] 	 0
	 [1.3585, 1.5039] 	 0
	 [1.3494, 1.5091] 	 0
	 [1.387, 1.4298] 	 1
	 [1.3105, 1.5004] 	 0
	 [1.395, 1.4622] 	 1
	 [1.3821, 1.4384] 	 1
	 [1.4249, 1.5253] 	 0
	 [1.4096, 1.4675] 	 1
	 [1.3625, 1.4853] 	 0
	 [1.364, 1.4371] 	 1
	 [1.3639, 1.5154] 	 0
	 [1.3803, 1.4571] 	 1
	 [1.3638, 1.5271] 	 0
	 [1.4373, 1.5033] 	 1
	 [1.4374, 1.4586] 	 1
	 [1.3349, 1.5287] 	 0
	 [1.399, 1.4043] 	 1
	 [1.2949, 1.5017] 	 0
	 [1.2943, 1.4934] 	 0
	 [1.3227, 1.4878] 	 0
	 [1.3281, 1.5408] 	 

	 [1.3508, 1.4935] 	 0
	 [1.295, 1.524] 	 0
	 [1.3624, 1.4956] 	 0
	 [1.4132, 1.4891] 	 0
	 [1.2872, 1.5016] 	 0
	 [1.4242, 1.4425] 	 1
	 [1.3694, 1.4237] 	 1
	 [1.3863, 1.4259] 	 1
	 [1.3507, 1.5113] 	 0
	 [1.3815, 1.4552] 	 1
	 [1.3316, 1.4768] 	 0
	 [1.4488, 1.4692] 	 1
	 [1.3429, 1.5034] 	 0
	 [1.4141, 1.4654] 	 1
	 [1.2795, 1.5434] 	 0
	 [1.3503, 1.4202] 	 1
	 [1.3232, 1.4715] 	 0
	 [1.4278, 1.5005] 	 1
	 [1.3612, 1.5271] 	 0
	 [1.3869, 1.4701] 	 1
	 [1.3513, 1.408] 	 1
	 [1.3007, 1.5163] 	 0
	 [1.2985, 1.4333] 	 1
	 [1.3567, 1.4877] 	 0
	 [1.363, 1.4857] 	 0
	 [1.4246, 1.498] 	 0
	 [1.3463, 1.4432] 	 1
	 [1.3636, 1.4378] 	 1
	 [1.3597, 1.5171] 	 0
	 [1.3581, 1.436] 	 1
	 [1.4505, 1.5081] 	 1
	 [1.3216, 1.4886] 	 0
	 [1.3325, 1.4462] 	 1
	 [1.3807, 1.4811] 	 1
	 [1.4049, 1.4401] 	 1
	 [1.3181, 1.4962] 	 0
	 [1.3859, 1.5056] 	 0
	 [1.3639, 1.4757] 	 0
	 [1.3155, 1.5267] 	 0
	 [1.3752, 1.5266] 	 0
	 [1.4125, 1.4625] 	 1
	 [1.339, 1.5385] 	 0
	 [1.384, 1.5146] 	 0
	 [1.3662, 1.5155] 

	 [1.3922, 1.4733] 	 1
	 [1.3675, 1.424] 	 1
	 [1.2745, 1.4883] 	 0
	 [1.4015, 1.5078] 	 0
	 [1.437, 1.519] 	 1
	 [1.3866, 1.4259] 	 1
	 [1.3134, 1.5055] 	 0
	 [1.3971, 1.462] 	 1
	 [1.2843, 1.4835] 	 0
	 [1.3671, 1.4835] 	 1
	 [1.3325, 1.5158] 	 0
	 [1.4435, 1.5036] 	 1
	 [1.3127, 1.4497] 	 1
	 [1.3831, 1.5229] 	 0
	 [1.3401, 1.49] 	 0
	 [1.3025, 1.5264] 	 0
	 [1.3655, 1.4937] 	 0
	 [1.4241, 1.4975] 	 0
	 [1.2949, 1.5049] 	 0
	 [1.4259, 1.4349] 	 1
	 [1.3748, 1.427] 	 1
	 [1.4035, 1.4347] 	 1
	 [1.3577, 1.5136] 	 0
	 [1.3756, 1.4523] 	 1
	 [1.3252, 1.4772] 	 0
	 [1.4549, 1.471] 	 1
	 [1.3421, 1.4997] 	 0
	 [1.4076, 1.4674] 	 1
	 [1.2979, 1.5425] 	 0
	 [1.3534, 1.4254] 	 1
	 [1.3315, 1.4771] 	 0
	 [1.4228, 1.4958] 	 1
	 [1.3542, 1.5205] 	 0
	 [1.3901, 1.4649] 	 1
	 [1.3529, 1.406] 	 1
	 [1.2951, 1.5158] 	 0
	 [1.3082, 1.4378] 	 1
	 [1.3565, 1.4845] 	 0
	 [1.364, 1.4897] 	 0
	 [1.4128, 1.4918] 	 0
	 [1.3483, 1.4459] 	 1
	 [1.3574, 1.4312] 	 1
	 [1.3506, 1.5145] 	 0
	 [1.3502, 1.4346] 	 

	 [1.396, 1.5273] 	 0
	 [1.3387, 1.4927] 	 0
	 [1.3008, 1.527] 	 0
	 [1.3661, 1.499] 	 0
	 [1.4211, 1.5008] 	 0
	 [1.3017, 1.5109] 	 0
	 [1.4261, 1.4402] 	 1
	 [1.3885, 1.4332] 	 1
	 [1.4063, 1.4421] 	 1
	 [1.3659, 1.5172] 	 0
	 [1.3788, 1.4529] 	 1
	 [1.3413, 1.4837] 	 0
	 [1.4584, 1.4775] 	 1
	 [1.3427, 1.4968] 	 0
	 [1.4075, 1.4682] 	 1
	 [1.2996, 1.5368] 	 0
	 [1.3456, 1.4238] 	 1
	 [1.3462, 1.4835] 	 0
	 [1.4285, 1.4947] 	 1
	 [1.3572, 1.5213] 	 0
	 [1.3928, 1.4558] 	 1
	 [1.3602, 1.4031] 	 1
	 [1.2972, 1.5179] 	 0
	 [1.3105, 1.4373] 	 1
	 [1.362, 1.4898] 	 0
	 [1.3666, 1.4923] 	 0
	 [1.4231, 1.4978] 	 0
	 [1.3411, 1.4449] 	 1
	 [1.3539, 1.4322] 	 1
	 [1.3532, 1.507] 	 0
	 [1.3481, 1.4333] 	 1
	 [1.4382, 1.5127] 	 1
	 [1.3276, 1.4878] 	 0
	 [1.3249, 1.4426] 	 1
	 [1.3936, 1.4811] 	 1
	 [1.3966, 1.447] 	 1
	 [1.3022, 1.4847] 	 0
	 [1.372, 1.493] 	 0
	 [1.3803, 1.4737] 	 0
	 [1.3191, 1.5238] 	 0
	 [1.3953, 1.5354] 	 0
	 [1.3859, 1.4511] 	 1
	 [1.3541, 1.5384] 	 0
	 [1.3945, 1.5171] 

	 [1.4232, 1.4368] 	 1
	 [1.3824, 1.4348] 	 1
	 [1.4022, 1.4414] 	 1
	 [1.3679, 1.516] 	 0
	 [1.3787, 1.4577] 	 1
	 [1.3447, 1.4826] 	 0
	 [1.4452, 1.463] 	 1
	 [1.3403, 1.4992] 	 0
	 [1.3959, 1.4598] 	 1
	 [1.3084, 1.5353] 	 0
	 [1.3565, 1.4236] 	 1
	 [1.348, 1.4855] 	 0
	 [1.4201, 1.4912] 	 1
	 [1.3769, 1.5277] 	 0
	 [1.3941, 1.4501] 	 1
	 [1.3659, 1.4037] 	 1
	 [1.289, 1.5116] 	 0
	 [1.3302, 1.4422] 	 1
	 [1.383, 1.5016] 	 0
	 [1.3707, 1.4985] 	 0
	 [1.4341, 1.5072] 	 0
	 [1.3361, 1.4455] 	 1
	 [1.3472, 1.4285] 	 1
	 [1.3606, 1.51] 	 0
	 [1.3447, 1.43] 	 1
	 [1.4342, 1.5056] 	 1
	 [1.332, 1.4816] 	 0
	 [1.3309, 1.4429] 	 1
	 [1.3918, 1.4809] 	 1
	 [1.41, 1.4608] 	 1
	 [1.3142, 1.4861] 	 0
	 [1.376, 1.498] 	 0
	 [1.38, 1.48] 	 0
	 [1.3314, 1.5272] 	 0
	 [1.4071, 1.5408] 	 0
	 [1.3964, 1.457] 	 1
	 [1.3583, 1.5422] 	 0
	 [1.3879, 1.5115] 	 0
	 [1.3876, 1.5196] 	 0
	 [1.429, 1.4573] 	 1
	 [1.3198, 1.5015] 	 0
	 [1.4119, 1.4653] 	 1
	 [1.3784, 1.4492] 	 1
	 [1.4264, 1.5065] 	 0
	 [1.408

In [172]:
print("\nFlattened gradient angle matrix for all rounds\n")
X, y = [],[]
print("Round:",end=" ")
for r in range(len(data)):
    print(r,end=", ")
    X_flat = flatten_layers(data[r])
    matrix = flattened_round_angle_matrix(X_flat)
    for i,row in enumerate(matrix):
        #del row[i]
        X.append(row)
        y.append(y_flat[i])
print("X : (",len(X),"x",len(X[0]),")")
print("y : (",len(y),"x",1,")")


Flattened gradient angle matrix for all rounds

Round: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, X : ( 10000 x 100 )
y : ( 10000 x 1 )


In [167]:
# #Angle similarity for layer for all rounds, create angle matrix dataset and labels
# print("\nConv1 gradient angle matrix for all rounds\n")
# X, y = [],[]
# for r in range(len(data)):
#     print("round: ",r)
#     matrix = get_round_angle_matrix(data[r],'conv1')
#     #matrix = get_round_cos_sim_matrix(data[0],'conv1')
#     for i,row in enumerate(matrix):
#         #del row[i]
#         X.append(row)
#         y.append(data[r][i][1])
#         print("\t",X[-1])

In [108]:
#Format data
import torch
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

print(Counter(y))
# ros = RandomOverSampler()
# X, y = ros.fit_resample(X, y)
print(Counter(y))
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=12)

X_train = torch.from_numpy(np.array(X_train).astype(np.float32))
y_train = torch.from_numpy(np.array(y_train).astype(np.float32))
X_test = torch.from_numpy(np.array(X_test).astype(np.float32))
y_test = torch.from_numpy(np.array(y_test).astype(np.float32))

y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

Counter({1: 5000, 0: 5000})
Counter({1: 5000, 0: 5000})


In [109]:
print("Train size: ",len(y_train), " - (", len([1 for d in y_train if d == 0]),":",len([1 for d in y_train if d == 1]),")")
print("Test size: ",len(y_test), " - (", len([1 for d in y_test if d == 0]),":",len([1 for d in y_test if d == 1]),")")

Train size:  8000  - ( 4019 : 3981 )
Test size:  2000  - ( 981 : 1019 )


In [110]:
#GradNet model for classifying gradients
import torch
import torch.nn as nn
import torch.nn.functional as F

# Hyper-parameters 
num_epochs = 80000
batch_size = 10
learning_rate = 0.01

class GradNet(nn.Module):
    
    def __init__(self,input_size):
        super(GradNet,self).__init__()
        self.linear1 = nn.Linear(input_size,64)
        self.linear2 = nn.Linear(64,32)
        self.linear3 = nn.Linear(32,32)
        self.linear4 = nn.Linear(32,1)
        
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = F.relu(self.linear3(X))
        X = self.linear4(X)
        return torch.sigmoid(X)

model = GradNet(X_train.shape[1])
weight = torch.tensor([0.2])
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [111]:
#Training
for epoch in range(num_epochs):
    # Forward pass
    y_pred = model(X_train)

    #loss = criterion(y_pred, y_train.reshape((y_train.shape[0],1)))
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (epoch + 1) % 5000 == 0:
        print(f'epoch: {epoch+1} \t loss = {loss.item()}')
        


epoch: 5000 	 loss = 0.6767414808273315
epoch: 10000 	 loss = 0.49227219820022583
epoch: 15000 	 loss = 0.4779326319694519
epoch: 20000 	 loss = 0.46131041646003723
epoch: 25000 	 loss = 0.44844457507133484
epoch: 30000 	 loss = 0.4423781633377075
epoch: 35000 	 loss = 0.43844085931777954
epoch: 40000 	 loss = 0.4289311170578003
epoch: 45000 	 loss = 0.4241653382778168
epoch: 50000 	 loss = 0.42239224910736084
epoch: 55000 	 loss = 0.419467031955719
epoch: 60000 	 loss = 0.4182458221912384
epoch: 65000 	 loss = 0.41630133986473083
epoch: 70000 	 loss = 0.4141080379486084
epoch: 75000 	 loss = 0.4153163731098175
epoch: 80000 	 loss = 0.41386231780052185


In [112]:
#Evaluation
from sklearn.metrics import confusion_matrix

def performance(X_test,y_test):
    with torch.no_grad(): #Similar to detach, defuses gradient storage for the model
        y_pred = model(X_test)
        y_pred_classes = y_pred.round() #round preds to 0 or 1
        acc = y_pred_classes.eq(y_test).sum() / float(y_test.shape[0])
        print(confusion_matrix(y_test, y_pred_classes))
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred_classes).ravel()
        print("TP: ",tp)
        print("TN: ",tn)
        print("FP: ",fp)
        print("FN: ",fn)
        print("Precision: ", tp / (tp + fp))
        print("Recall: ", tp / (tp + fn))
        print(f'Acc: {acc:.4f}')

print("+++ Train +++\n")
performance(X_train,y_train)
print("\n+++ Test +++\n")
performance(X_test,y_test)

+++ Train +++

[[3396  623]
 [ 838 3143]]
TP:  3143
TN:  3396
FP:  623
FN:  838
Precision:  0.8345724907063197
Recall:  0.7895001255965838
Acc: 0.8174

+++ Test +++

[[818 163]
 [175 844]]
TP:  844
TN:  818
FP:  163
FN:  175
Precision:  0.8381330685203575
Recall:  0.8282630029440629
Acc: 0.8310


In [115]:
from sklearn.cluster import KMeans

def measure(y_pred,y_test):
    with torch.no_grad(): #Similar to detach, defuses gradient storage for the model
        #y_pred_classes = y_pred.round() #round preds to 0 or 1
        acc = y_pred.eq(y_test).sum() / float(y_test.shape[0])
        print(confusion_matrix(y_test, y_pred))
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        print("TP: ",tp)
        print("TN: ",tn)
        print("FP: ",fp)
        print("FN: ",fn)
        print("Precision: ", tp / (tp + fp))
        print("Recall: ", tp / (tp + fn))
        #print(f'Acc: {acc:.4f}')
        print(f'Acc: {(tp + tn)/(tp+tn+fp+fn):.4f}')

kmeans = KMeans(n_clusters=2,random_state=4).fit(torch.from_numpy(np.array(X)))
y_pred = torch.from_numpy(np.array(kmeans.labels_).astype(np.float32))
measure(y_pred,torch.from_numpy(np.array(y)))

[[2631 2369]
 [3914 1086]]
TP:  1086
TN:  2631
FP:  2369
FN:  3914
Precision:  0.31432706222865414
Recall:  0.2172
Acc: 0.3717


In [120]:
_0_avg_theta1, _1_avg_theta1, _0_avg_theta2, _1_avg_theta2 = 0,0,0,0
for i,val in enumerate(X):
    if y[i] == 0:
        _0_avg_theta1 += X[i][0]
        _0_avg_theta2 += X[i][1]
    else:
        _1_avg_theta1 += X[i][0]
        _1_avg_theta2 += X[i][1]
print("class 0: \ttheta 1: ", np.degrees(_0_avg_theta1 / 5000), "\ttheta 2: ", np.degrees(_0_avg_theta2 / 5000))
print("class 1: \ttheta 1: ", np.degrees(_1_avg_theta1 / 5000), "\ttheta 2: ", np.degrees(_1_avg_theta2 / 5000))

class 0: 	theta 1:  75.56559992980398 	theta 2:  85.50699177799527
class 1: 	theta 1:  78.14579045208743 	theta 2:  82.95953891712625


In [310]:
with open("classifier.pkl","wb") as f:
    pickle.dump(model,f)